In [4]:
from n2v_predict import predict_n2v
from envutils import load_env, ENV
from pathlib import Path


load_env(".env_vdi", 
         parser_args= {
            "DATASET_FOLDER": "/localscratch/calcium_imaging_dataset/calcium_imaging/NIDDL/",
         })

EXPERIMENT_NAME = "NIDDL_N2V_4"
DATASET_NAME = "Low_SNR"
GT_DATASET_NAME = "High_SNR"

In [5]:
import tifffile
import pandas as pd
from careamics.utils.metrics import scale_invariant_psnr


results = pd.DataFrame()

for high_snr_path in Path(ENV["DATASET_FOLDER"]).joinpath(GT_DATASET_NAME).glob("*.tif*"):
    low_snr_path = Path(str(high_snr_path).replace(GT_DATASET_NAME, DATASET_NAME).replace("200ms", "20ms"))
    pred_path = Path(ENV["OUTPUT_FOLDER"]).joinpath(EXPERIMENT_NAME, low_snr_path.name)

    high_snr = tifffile.imread(high_snr_path)
    low_snr = tifffile.imread(low_snr_path)
    pred = tifffile.imread(pred_path)

    gt_psnr = scale_invariant_psnr(gt=high_snr, pred=low_snr)
    pred_psnr = scale_invariant_psnr(gt=high_snr, pred=pred)
    new_row = pd.DataFrame({"filename": low_snr_path.name, "Low_SNR_PSNR": gt_psnr, "Pred_PSNR": pred_psnr}, index=[0])
    results = pd.concat([results, new_row], ignore_index=True)

results["PSNR_diff"] = results["Pred_PSNR"] - results["Low_SNR_PSNR"]
results.to_csv(f'results/{EXPERIMENT_NAME}.csv')

In [6]:

mean_values = results[['Low_SNR_PSNR', 'Pred_PSNR', 'PSNR_diff']].mean()
std_values = results[['Low_SNR_PSNR', 'Pred_PSNR', 'PSNR_diff']].std()

summary_df = pd.DataFrame([mean_values, std_values], index=['mean', 'std'])

print(summary_df)

      Low_SNR_PSNR  Pred_PSNR  PSNR_diff
mean     27.589894  30.534130   2.944237
std       2.124671   2.707268   1.854985
